In [204]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [205]:
digits = load_digits()

In [206]:
digits.keys()

dict_keys(['data', 'target', 'frame', 'feature_names', 'target_names', 'images', 'DESCR'])

In [207]:
print(digits.target)
print(digits.data)
print(digits.feature_names)

[0 1 2 ... 8 9 8]
[[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]
['pixel_0_0', 'pixel_0_1', 'pixel_0_2', 'pixel_0_3', 'pixel_0_4', 'pixel_0_5', 'pixel_0_6', 'pixel_0_7', 'pixel_1_0', 'pixel_1_1', 'pixel_1_2', 'pixel_1_3', 'pixel_1_4', 'pixel_1_5', 'pixel_1_6', 'pixel_1_7', 'pixel_2_0', 'pixel_2_1', 'pixel_2_2', 'pixel_2_3', 'pixel_2_4', 'pixel_2_5', 'pixel_2_6', 'pixel_2_7', 'pixel_3_0', 'pixel_3_1', 'pixel_3_2', 'pixel_3_3', 'pixel_3_4', 'pixel_3_5', 'pixel_3_6', 'pixel_3_7', 'pixel_4_0', 'pixel_4_1', 'pixel_4_2', 'pixel_4_3', 'pixel_4_4', 'pixel_4_5', 'pixel_4_6', 'pixel_4_7', 'pixel_5_0', 'pixel_5_1', 'pixel_5_2', 'pixel_5_3', 'pixel_5_4', 'pixel_5_5', 'pixel_5_6', 'pixel_5_7', 'pixel_6_0', 'pixel_6_1', 'pixel_6_2', 'pixel_6_3', 'pixel_6_4', 'pixel_6_5', 'pixel_6_6', 'pixel_6_7', 'pixel_7_0', 'pixel_7_1', 'pixel_7_2', 'pixel_7_3', 'pixel_7_

In [208]:
X,y = digits.data, digits.target

In [209]:
X_train,X_test,y_train, y_test = train_test_split(X,y, test_size= 0.2, random_state=32)

In [210]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [211]:
class Custom_dataset(Dataset):
    def __init__(self,data,target):
        self.data = torch.tensor(data, dtype=torch.float32)
        self.target = torch.tensor(target, dtype =torch.long)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        sample = {'data' : self.data[index], 'target' : self.target[index]}
        return sample

In [212]:
train_dataset = Custom_dataset(X_train,y_train)
test_dataset = Custom_dataset(X_test,y_test)

In [213]:
len(train_dataset)

1437

In [214]:
len(test_dataset)

360

In [215]:
train_dataloader = DataLoader(dataset = train_dataset, batch_size= 32, shuffle = True)
test_dataloader = DataLoader(dataset = test_dataset, batch_size=32, shuffle =False)

In [216]:
class SimpleNN(nn.Module):
    def __init__(self,input_size,output_size):
        super(SimpleNN,self).__init__()
        self.ann = nn.Sequential(
            nn.Linear(input_size, 23),
            nn.ReLU(),
            nn.Linear(23,38),
            nn.ReLU(),
            nn.Linear(38, output_size)
        )
        
    def forward(self,x):
        x = self.ann(x)
        return x

In [217]:
X_train[0].shape

(64,)

In [218]:
X_train[0]

array([ 0.        , -0.3210702 , -0.01242153, -0.39509689, -0.2125568 ,
        1.28518375,  3.17297064, -0.12708867, -0.05649233,  0.02735604,
        0.67899414, -1.03215927, -0.49703043,  0.80914171,  2.21328684,
       -0.13444943, -0.04991522, -0.44179191, -0.70936563, -1.21461872,
       -0.48673157,  0.83548772, -0.55226769, -0.12011988, -0.03733267,
       -0.78647613, -1.48894968, -1.32467066,  0.82950304, -0.96013651,
       -0.62844139, -0.0528332 ,  0.        , -0.67216495, -1.23788121,
        0.29146399,  0.78464197, -0.11998762, -0.51988925,  0.        ,
       -0.06519029,  0.13944204,  1.22789725,  1.18620546,  0.03504463,
       -0.20374872, -0.78034166, -0.09192682, -0.03963009,  0.18249972,
        0.26787874,  0.44637237, -1.80413333, -1.4315759 , -0.74320923,
       -0.20110147, -0.02638899, -0.29746521,  0.32605198, -0.21120501,
       -2.40662136, -1.13368971, -0.50143795, -0.19605754])

In [219]:
len(set(y_train))

10

In [220]:
input_size = X_train.shape[1]
output_size = len(set(y_train))
lr = 0.001
epochs = 30

In [221]:
model = SimpleNN(input_size,output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

## Training Loop

In [222]:
from tqdm import tqdm
from torchmetrics import Accuracy

In [223]:
for epoch in range(epochs):
    loop = tqdm(enumerate(train_dataloader), total=len(train_dataloader))
    model.train()
    running_loss = 0.0
    for idx, batch in loop:
        inputs = batch['data']
        targets = batch['target']
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs,targets)
        loss.backward()
        optimizer.step()
        acc = Accuracy(task="multiclass", num_classes=output_size)
        accuracy = acc(outputs, targets)
        # running_loss += running_loss.item()
        loop.set_postfix(loss= loss.item(), accuracy= accuracy.item())
        loop.set_description(f"Epoch : [{epoch}/{epochs}]")
        
    print(f"Epoch{epoch+1}/{epochs}, Loss :{running_loss}/len({train_dataloader})")

Epoch : [0/30]: 100%|██████████| 45/45 [00:00<00:00, 550.03it/s, accuracy=0.552, loss=1.99]


Epoch1/30, Loss :0.0/len(<torch.utils.data.dataloader.DataLoader object at 0x303d83160>)


Epoch : [1/30]: 100%|██████████| 45/45 [00:00<00:00, 502.88it/s, accuracy=0.69, loss=1.24]


Epoch2/30, Loss :0.0/len(<torch.utils.data.dataloader.DataLoader object at 0x303d83160>)


Epoch : [2/30]: 100%|██████████| 45/45 [00:00<00:00, 503.52it/s, accuracy=0.828, loss=0.675]


...

Epoch28/30, Loss :0.0/len(<torch.utils.data.dataloader.DataLoader object at 0x303d83160>)


Epoch : [28/30]: 100%|██████████| 45/45 [00:00<00:00, 614.77it/s, accuracy=1, loss=0.031]


Epoch29/30, Loss :0.0/len(<torch.utils.data.dataloader.DataLoader object at 0x303d83160>)


Epoch : [29/30]: 100%|██████████| 45/45 [00:00<00:00, 586.06it/s, accuracy=1, loss=0.00823]

Epoch30/30, Loss :0.0/len(<torch.utils.data.dataloader.DataLoader object at 0x303d83160>)


## Evaluation Loop

In [228]:
model.eval()
all_predictions = []
all_targets =[]

with torch.no_grad():
    for batch in test_dataloader:
        input = batch['data']
        targets =batch['target']
        
        outputs = model(inputs)
        predictions = torch.argmax(outputs, dim =1)
        
        all_predictions.extend(predictions.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())
        acc = accuracy_score(all_predictions,all_targets)
        print(f"Accracy Test : {acc *100: .3f}%")

ValueError: Found input variables with inconsistent numbers of samples: [29, 32]